## Librerias y dependencias
Se instalaron las bibliotecas necesarias para el procesamiento de lenguaje natural, el uso del modelo, la evaluacion de este y la manipulacion de datos en general, entre ellos: `datasets, transformers, accelerate, optimum, auto-gptq, rouge_score`

In [ ]:
!pip install datasets # For download datasets from kaggle
!pip install transformers accelerate optimum # some libs for transformers and optimization
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu117/ #GPU accelerated package for gpt
!pip install rouge_score # for scoring
!pip install nltk # the classical one
!pip install kagglehub # for download datasets from kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# IA
import torch
# Models
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
# DataManipulation
import pandas as pd
# General
import os
# Datasets
import kagglehub
from datasets import load_dataset
# For widgets
import ipywidgets as widgets
from IPython.display import display, clear_output

## Funcionalidades Generales

In [ ]:
def load_data():
  """
  Load the Colombian Spanish News dataset preprocessed and loaded into kaggle

  Returns:
  - dataset: The loaded dataset.
  """

  # Download latest version
  path = kagglehub.dataset_download("sebastianarangou/just-some-preprocessed-data-for-homework")

  print("Path to dataset files:", path)
  # Assuming the dataset is in CSV format
  df_train_path = path + '/df_train.csv'  # Adjust based on the actual file name
  df_test_path = path + '/df_test.csv'  # Adjust based on the actual file name
  df_val_path = path + '/df_val.csv'  # Adjust based on the actual file name

  # Load the dataset into a Pandas DataFrame
  df_train = pd.read_csv(df_train_path)
  df_test = pd.read_csv(df_test_path)
  df_val = pd.read_csv(df_val_path)

  return df_train, df_test, df_val

In [ ]:
def load_data_not_preprocessed():
  """
  Load the Colombian Spanish News dataset.

  Returns:
  - dataset: The loaded dataset.
  """
  # Load the Colombian Spanish News dataset
  dataset = load_dataset("hacktoberfest-corpus-es/colmbian_spanish_news")

  # Access the dataset split (adjust as necessary)
  train_data = dataset["train"]
  test_data = dataset["test"]

  return train_data, test_data


In [ ]:
def process_csv(filepath):
  """Reads a CSV, processes each row, and optionally saves the results."""
  try:
    df = pd.read_csv(filepath, encoding='utf-8') # Try UTF-8 first
  except UnicodeDecodeError:
    print(f"Error reading CSV: {UnicodeDecodeError}")
    return

  resumenes = []
  categorias = []
  temas = []
  for index, noticia in df.iterrows():
    titulo = noticia["titulo"]
    texto = noticia["texto"]

    categoria = get_category(titulo, texto)
    temas_categoria = get_topics(categoria, titulo, texto)
    resumen = summarize_text(texto)

    categorias.append( categoria )
    temas.append( temas_categoria )
    resumenes.append( resumen )

  return categorias, temas, resumenes

## Modulo categorias

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Cargar modelo y tokenizador
model_path = "/content/drive/Shareddrives/PLN/Practica02/Modulo categorias/Modelo_categorias_bert"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Asegurar que el modelo está en la GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Diccionario con las categorias
label_dict = {
      'gente': 0,
      'innovación': 1,
      'macroeconomia': 2,
      'alianzas': 3,
      'reputacion': 4,
      'regulaciones': 5,
      'otros': 6,
      'sostenibilidad': 7
  }

id2category = {idx: cat for cat, idx in label_dict.items()}

def get_category(titulo, texto):
  """
  This function helps to get the category of a news.
  params:
  - titulo: the title of the news
  - texto: the text of the news
  returns:
  - the category of the news
  """
  inputs = tokenizer(titulo,texto, return_tensors="pt", truncation=True, max_length=128)
  inputs = inputs.to(device)  # Add this line to move inputs to the GPU
  with torch.no_grad():
      outputs = model(**inputs)
  logits = outputs.logits
  pred = torch.argmax(logits, dim=1).item()
  return id2category[pred]

In [ ]:
titulo_prueba = "Un nuevo avance tecnológico"
texto_prueba = "Este descubrimiento revoluciona la innovación en Europa."
print(get_category(titulo_prueba, texto_prueba))

innovación


## Modulo temas

In [ ]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 37.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install spacy nltk gensim

In [ ]:
import spacy
import nltk
from nltk.corpus import stopwords
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel

In [ ]:
# Cargar el diccionario
dictionary = Dictionary.load("/content/drive/Shareddrives/PLN/Practica02/Modulo_Temas/dictionary.gensim")

# Cargar el modelo LDA
lda_model = LdaModel.load("/content/drive/Shareddrives/PLN/Practica02/Modulo_Temas/lda_model.model")

# Número de palabras clave por tema
num_words = 11

# Cargar modelo de spaCy y stopwords de NLTK
nlp = spacy.load("es_core_news_sm", disable=["ner", "parser"])
nltk.download("stopwords")
stop_words = set(stopwords.words("spanish"))

def preprocess_text(news):
    doc = nlp(news)
    tokens = [token.lemma_ for token in doc if not token.is_punct and token.text.lower() not in stop_words and not token.is_digit]
    return tokens

def get_topics(news):
    tokens = preprocess_text(news)

    if not tokens:
        return None

    # Convierte el texto en bolsa de palabras
    bow = dictionary.doc2bow(tokens)

    # Obtiene la distribución de temas
    topics = lda_model.get_document_topics(bow)

    if not topics:
        return None

    sorted_topics = sorted(topics, key=lambda x: x[1], reverse=True)

    topics_in_words = []
    for topic_id, prob in sorted_topics:
        words = [word for word, _ in lda_model.show_topic(topic_id, num_words)]
        topics_in_words.append((topic_id, prob, words))

    # Mostrar tema y palabras clave
    for topic_id, prob, words in topics_in_words:
        print(f"Tema {topic_id}: {', '.join(words)}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
news = "La Economía Global Enfrenta Desafíos: Crecimiento Moderado y Riesgos Latentes. En un contexto de incertidumbre económica global, las principales economías del mundo están experimentando un crecimiento moderado mientras enfrentan desafíos como la inflación persistente, las políticas monetarias restrictivas y la desaceleración del comercio internacional. En este escenario, organismos multilaterales han ajustado sus proyecciones de crecimiento, advirtiendo sobre posibles riesgos a corto y mediano plazo."
get_topics(news)

Tema 3: precio, mercado, banco, economía, inflación, poder, económico, tasa, país, él, dólar
Tema 0: medio, ambiente, energía, agua, sostenibilidad, zona, año, proyecto, gas, ambiental, natural
Tema 5: empresa, él, poder, hacer, ser, nuevo, innovación, deber, tecnología, social, mundo
Tema 9: hacer, él, decir, red, dar, ver, ser, social, momento, gente, ir


## Modulo Resumenes

In [ ]:
def summarize_text(texto, show_summary=False):
  """
  This function helps to summarize a text
  params:
  - texto: the text to summarize
  - show_summary: if True, it will show the summary
  returns:
  - the summary of the text
  """
  # Constantes
  INITIAL_PROMPT:str = "Resume lo principal de este texto: "
  PROMPT_LEN:int = len(INITIAL_PROMPT)
  # Llamar al modelo y el autotokenizer
  tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
  model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small")

  # Check for GPU availability and move the model to the GPU if available
  if torch.cuda.is_available():
      device = torch.device("cuda")
      model.to(device)
  else:
      device = torch.device("cpu")
      print("No GPU available, model loaded to CPU")

  # Tokenizar el texto
  inputs = tokenizer.encode(
      "Resume lo principal de este texto: " + texto,
      return_tensors="pt",
      max_length=128,
      truncation=True
      ).to(device)

  # Generar el resumen
  resumen_tensor = model.generate(
      inputs,
      max_length=256,
      min_length=24,
      length_penalty=4.0,
      num_beams=4,
      early_stopping=True
      )

  # Decodificar
  resumen = tokenizer.decode(resumen_tensor[0], skip_special_tokens=True)
  clean_resumen = resumen[PROMPT_LEN:]

  # Print nice stuff
  resumen_ratio = len(clean_resumen)/len(texto)
  print(f'Resumen Ratio: {resumen_ratio}')

  if show_summary:
    print(f'Palabras: {len(clean_resumen)}')
    print(f'Resumen: \n {clean_resumen}')

  return clean_resumen

## Widgets


In [ ]:
# Botones

## Cargar CSV
upload_button = widgets.FileUpload(
    accept='.csv',
    description='Cargar CSV',
    multiple=False  # True to accept multiple files upload else False
    )
## Reset button
reset_button = widgets.Button(description="Reiniciar")

# Visualizar

## Categoria
category_output = widgets.Output()
## Resumen
summary_output = widgets.Output()
## Temas
topics_output = widgets.Output()
## Titulo
titulo_output = widgets.Output()

In [ ]:
def on_upload_change(change):
    if upload_button.value:
        #print(upload_button)
        # Get the first uploaded file
        #uploaded_file = list(upload_button.value.values())[0]
        # Decode the content of the uploaded file (assuming it's a CSV)
        byte_data = upload_button.value['prueba.csv']['content']
        decoded_content = byte_data.decode('utf-8')
        lines = decoded_content.split('\r\n')
        header = lines[0]
        data = lines[1]
        # Step 5: Extract "Titulo" and "Texto" from the data (remove quotes and decode special characters)
        titulo, texto = data.split('", "')
        # Clean up the quotes and decode special characters
        titulo = titulo.strip('"')
        texto = texto.strip('"')

        try:
            # Use the extracted values
            with category_output:
                categoria = get_category(titulo, texto)
                print(f'Categoria:\n{categoria}')
                print("-------------")

            with summary_output:
                resumen = summarize_text(texto)
                print(f'Resumen:\n{resumen}')
                print("-------------")

            with topics_output:
                topics = get_topics(news)
                print( topics )
                print("-------------")

        except Exception as e:
            with summary_output:
                clear_output()
                print(f"Error procesando el archivo: {e}")
upload_button.observe(on_upload_change, names='value')

def on_reset_button_clicked(b):
    upload_button.value.clear()  # Clear the upload button's value
    with category_output:
        clear_output()
    with summary_output:
        clear_output()
    with topics_output:
        clear_output()

reset_button.on_click(on_reset_button_clicked)

### Aplicacion

In [ ]:
# Display widgets
# Csv de prueba:
  # Titulo: Antioquia creció 2,5%, creó casi 30.000 empresas y tuvo mejor desempeño que el promedio nacional en 2024
  # Texto: El 2024 fue de estabilización para la economía de Antioquia, pues sectores claves de la producción como la industria, el comercio y la construcción que venían con indicadores negativos repuntaron, aunque sin recuperarse
  #        totalmente.Así lo señaló la Cámara de Comercio de Medellín para Antioquia, entidad que presentó el informe sobre el desempeño económico reciente del departamento, en el que se evidenció que los principales sectores
  #        cerraron 2024 con mejores cifras respecto a 2023, particularmente el turismo y las ventas el exterior.Aunque la industria local permaneció en terreno negativo, los indicadores de producción y ventas manufactureras
  #        reflejaron menores ritmos de desaceleración. Sin embargo, se requieren estrategias de reactivación que impulsen la dinámica en 2025.El panorama del sector construcción siguió mostrando rezagos. El área licenciada cayó 23%,
  #        cifra mayor al 19% reportado para el año 2023.Los despachos de cemento gris, a diciembre, también siguieron cayendo (-3,7%), aunque esta vez 1,8 puntos porcentuales por debajo del consolidado nacional.Además, persistió la baja dinámica del mercado de vivienda. En Antioquia las ventas cayeron más que en el consolidado de las 19 regionales de Camacol (donde se apreció una leve mejora de 0,6%). La Vivienda de Interés Social (VIS) bajó 25,6%, y la No VIS se contrajo 10,6%.Los sectores más dinámicos en AntioquiaFredy Pulgarín Sierra, vicepresidente de Desarrollo Empresarial y Competitividad de la Cámara de Comercio de Medellín, destacó algunos sectores que tuvieron un desempeño positivo.“Hay actividades que vienen creciendo bastante bien, y jalonan mucho el crecimiento económico. Por ejemplo, el turismo que crece muy bien y con un aumento de visitantes extranjeros en una proporción muy importante. Las exportaciones siguen creciendo bien, al 11%, impulsadas por oro, banano y flores”, mencionó.Estas conclusiones están alineadas con las estimaciones del Centro de Pensamiento de la Universidad EIA,
  #        que recientemente, señaló que el crecimiento de la economía regional estuvo estimulado por un año de buen comportamiento de las exportaciones, y el turismo, que tuvo un aumento de 23,9% en visitantes extranjeros no residentes.

display(upload_button, reset_button, category_output, summary_output, topics_output)

FileUpload(value={}, accept='.csv', data=[b'Titulo, Texto\r\n"Antioquia creci\xc3\xb3 2,5%, cre\xc3\xb3 casi 3…

Button(description='Reiniciar', style=ButtonStyle())

Output()

Output()

Output()

In [ ]:
import io
def read_csv(change):
    # Obtener el archivo cargado
    uploaded_file = list(upload_button.value.values())[0]
    content = uploaded_file['content']

    # Leer el contenido del archivo CSV
    df = pd.read_csv(io.BytesIO(content))
    print(df.head())  # Mostrar las primeras filas del DataFrame
# Conectar la función al widge
upload_button.observe(read_csv, names='value')